# import các thư viện

In [1]:
import cv2
import torch
import numpy as np
from skimage import measure
import os
import pandas as pd
import matplotlib.pyplot as plt


c:\Users\dangt\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Dùng mô hình có tệp best.pt train được bằng YoloV5**

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')

Using cache found in C:\Users\dangt/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-20 Python-3.10.0 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


**Hàm xuất ra ma trận nhị phân đặc trưng cho kí tự**

In [3]:
def feature(folder_path, file):
    binaries = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            # Đọc tệp tin ảnh
            img = cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_GRAYSCALE)

            # Đổi ảnh sang ma trận nhị phân
            ret, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

            # Thêm ma trận nhị phân vào mảng
            binaries.append(binary)
        

    # Khởi tạo mảng pixel trắng đen có kích thước giống với các ma trận trong binaries với các phần tử được khởi tạo là 0
    black_pixels = np.zeros_like(binaries[0])
    white_pixels = np.zeros_like(binaries[0])
    
    # Tính toán số lượng pixel màu đen và màu trắng cho từng pixel trong các ma trận
    for binary in binaries:
        black_pixels += binary == 0
        white_pixels += binary == 255

    # So sánh số lượng pixel màu đen và màu trắng để quyết định màu sắc cuối cùng của pixel trong ma trận đặc trưng
    feature = np.zeros_like(binaries[0], dtype=np.uint8)
    for i in range(feature.shape[0]):
        for j in range(feature.shape[1]):
            if black_pixels[i][j] > white_pixels[i][j]:
                feature[i][j] = 0
            else:
                feature[i][j] = 255

    return feature

**Xuất ra ma trận đặc trưng cho tất cả các kí tự**

In [4]:
features = []
folder_path = "char/"
files = os.listdir(folder_path)
for file in files:
    if os.path.isdir(os.path.join(folder_path, file)):
        folder_path_ii = os.path.join(folder_path, file)
        features.append(feature(folder_path_ii, file))

In [5]:
features2 = []
folder_path = "char2line/"
files = os.listdir(folder_path)
for file in files:
    if os.path.isdir(os.path.join(folder_path, file)):
        folder_path_ii = os.path.join(folder_path, file)
        features2.append(feature(folder_path_ii, file))

**So sánh để đưa ra kết luận**

In [6]:
def predict(binary, number, features):
    n = 0
    max = 0
    i = 0

    # number == 1: kí tự đang đọc là kí tự chữ số
    if (number == 1):

        # Đọc từ 0 đến 9
        for i in range (10):

            # Tính số pixel giống nhau của chữ số cần dự đoán và đặc trưng của các chữ số
            matching_pixels = np.sum(binary == features[i])
            total_pixels = binary.shape[0] * binary.shape[1]

            # Tính % giống nhau bằng cách lấy số pixel giống nhau / tổng số pixel * 100
            matching_percentage = matching_pixels / total_pixels * 100

            # Lấy max = tỉ lệ khớp nhất của chữ số cần dự đoán với các chữ số đặc trưng, n là chữ số được dự đoán
            if (matching_percentage>max): 
                n = i
                max = matching_percentage

    # number == 0: kí tự đang đọc là kí tự chữ cái
    else:

        # Nếu không phải chữ số thì đọc các feature chữ cái
        for i in range(10, len(features)):
            matching_pixels = np.sum(binary == features[i])
            total_pixels = binary.shape[0] * binary.shape[1]
            matching_percentage = matching_pixels / total_pixels * 100
            if (matching_percentage>max): 
                n = i
                max = matching_percentage

    if (n == 7 and max < 65): 
        n = 1

    return n, max

**Hàm đọc kí tự**

In [7]:
def read_char(labels_list, max, type):
    
    # Xếp kí tự theo thứ tự từ trái sang phải
    sorted_list = sorted(labels_list, key=lambda x: x[0])
    
    # Nếu số vùng kết nối lớn hơn số kí tự tối đa hiện có trên biển số xe thì sẽ có nhiễu , do đó đặt biến xóa = tổng số kí tự đang có - số kí tự tối đa có thể có
    if (len(labels_list) > max):
        delete = len(labels_list) - max
    else:
        delete = 0
        
    predicted = []
    
    # Type == 0: đây là biển 1 dòng, gồm có 7-8 kí tự, trong đó có kí tự chữ nằm ở vị trí thứ 3 từ trái sang phải
    # Type == 1: đây là biển 2 dòng, gồm có 7-8 kí tự, trong đó kí tự chữ nằm ở vị trí thứ 3 dòng trên, biển chia làm 2 dòng, dòng trên có 3 kí tự, dòng dưới 4-5 kí tự
    # Đọc từng kí tự từ trái sang phải, lưu vào mảng dự đoán gồm thứ tự, dự đoán nếu là chữ, dự đoán nếu là số
    if (type == 0):
        for i in range(len(sorted_list)):
            x, dg, y = sorted_list[i]
            predicted.append((i, predict(dg, 1, features), predict(dg, 0, features)))
    
    else:
        for i in range(len(sorted_list)):
            x, dg, y = sorted_list[i]
            predicted.append((i, predict(dg, 1, features2), predict(dg, 0, features2)))
            
    # Xếp lại dãy dự đoán theo thứ tự giảm dần của độ chính xác nhằm loại bỏ nhiễu có ánh sáng tốt + kích thước ngang với kí tự nhưng có 
    # tỉ lệ giống các đặc trưng khác rất thấp
    predicted = sorted(predicted, key=lambda x: x[1][1], reverse=True)
    predicted_string = ""
    j = 0
    lst = [list(item) for item in predicted]
    
    # Nếu là biển số 1 dòng thì xem thử nó có khả năng là chữ nên độ chính xác so với số thấp không, nếu không thì bỏ nó ra khỏi mảng
    # Nếu nó có khả năng là chữ nên match với số thấp thì check xem độ match của nó với chữ, vùng kết nối nào match với chữ và số đều thấp thì loại
    if (type == 0):
        for i in reversed(range(len(predicted))):
            if (j == delete):
                break
            if (lst[i][0] < 2):
                lst.remove(lst[i])
                j = j + 1
            elif (lst[i][0] > (2+ delete)):
                lst.remove(lst[i])
                j = j + 1
            else:
                if (lst[i][2][1] >= 65):
                    lst.remove(lst[i-1])
                    j = j + 1
                else:
                    lst.remove(lst[i])
                    j = j + 1
    if (type == 1):
        for i in reversed(range(len(predicted))):
            if (j == delete):
                break
            if (lst[i][0] < 2):
                lst.remove(lst[i])
                j = j + 1
            else:
                if (lst[i][2][1] >= 65):

                    lst.remove(lst[i-1])
                    j = j + 1
                else:

                    lst.remove(lst[i])
                    j = j + 1
    if (type == 2):
        if (delete != 0):
            for i in range (delete):
                lst.pop(-1)

    predicted = [tuple(item) for item in lst] # chuyển lại thành tuple
    i = 0
    
    # Nối các số + chữ dự đoán được vào chuỗi số, đọc kí tự từ trái sang phải
    for item in sorted(lst, key=lambda x: x[0]):
            i = i + 1

            if (i == 3 and type == 0):
                if (item[2][0] == 10):
                    predicted_string += "A"
                if (item[2][0] == 11):
                    predicted_string += "B"
                if (item[2][0] == 12):
                    predicted_string += "C"
                if (item[2][0] == 13):
                    predicted_string += "D"
                if (item[2][0] == 14):
                    predicted_string += "F"
                if (item[2][0] == 15):
                    predicted_string += "G"
                if (item[2][0] == 16):
                    predicted_string += "L"
                if (item[2][0] == 17):
                    predicted_string += "N"
                if (item[2][0] == 18):
                    predicted_string += "S"
                if (item[2][0] == 19):
                    predicted_string += "V"
                if (item[2][0] == 20):
                    predicted_string += "Y"
                if (item[2][0] == 21):
                    predicted_string += "Z"
                if (item[2][0] == 22):
                    predicted_string += "E"
            elif (i==3 and type == 1):
                if (item[2][0] == 10):
                    predicted_string += "A"
                if (item[2][0] == 11):
                    predicted_string += "E"
                if (item[2][0] == 12):
                    predicted_string += "G"
                
            else:
                predicted_string += str(item[1][0])
                
    # Return chuỗi dự đoán được
    return predicted_string

In [8]:
def read(cropped_image, type):
    gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    # Sử dụng hàm connectedComponents để tìm các vùng kết nối trên ảnh nhị phân
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    num_labels, labels = cv2.connectedComponents(binary)
        
    d = 0
        # Sử dụng hàm connectedComponentsWithStats để tính toán diện tích của các vùng kết nối
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
    for i in range(num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area < 90 or area > 1500:
            labels[labels == i] = 0
        else: 
            d = d+1
    
    x_centroids = centroids[:, 0]
    y_centroids = centroids[:, 1]
    
    labels_list = []

    if (d > 5):

        for i in range(1, num_labels):
            # Lấy thông tin của nhãn hiện tại
            x, y, w, h, area = stats[i]

            # Nếu diện tích của nhãn nằm trong khoảng từ 100 đến 1500
            if 90 < area < 1500:
                # Cắt ảnh của nhãn hiện tại từ ảnh gốc
                digit_img = binary[y:y+h, x:x+w]
                digit_img = cv2.resize(digit_img, (25,60))
                # cv2.imshow(f"Digit {i}", digit_img)
                # cv2.waitKey()
                # cv2.destroyAllWindows()
                labels_list.append((x_centroids[i], digit_img, y_centroids[i]))      
    else:
        return ""
    
    read_final = None
    if (type == 1):
        max = 8
        read_final = read_char(labels_list, max, 0)
    else: 
        label_distances = []
        first_label = min(labels_list, key=lambda label: label[0])
        first_x_centroid, first_digit_img, first_y_centroid = first_label
        predicted = predict(first_digit_img, 1,  features2)
        while (len(labels_list) > 8):
            if (predicted[1] < 65):
                for i, label in enumerate(labels_list):
                    if label == first_label:
                        del labels_list[i]
                        break
                first_label = min(labels_list, key=lambda label: label[0])
                first_x_centroid, first_digit_img, first_y_centroid = first_label
                predicted = predict(first_digit_img, 1, features2)
            else:
                break
            
        line1 = []
        line2 = []
        i = 0
        labels_list.sort(key=lambda x: x[0])

        for label in labels_list:
            x_centroid, digit_img, y_centroid = label
            distance = abs(y_centroid - first_y_centroid)
            distance_x = abs(x_centroid - first_x_centroid)
            label_distances.append(distance)
            if (distance >= 6 + i*1.5):
                line1.append(label)  
            else:
                line2.append(label)
            i = i + 1

        max_line1 = 3
        max_line2 = 5
        line1 = read_char(line1, max_line1, 1)
        line2 = read_char(line2, max_line2, 2)
        read_final = line1 + line2

    return read_final

# đọc ký tự trên biển số xe

In [9]:
def readPlate(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image)

    for i, det in enumerate(results.xyxy[0]):
        type = 0
        
        # Lấy tọa độ bbox của vật thể thứ i
        bbox = det[0:4].cpu().numpy()

        # Cắt lấy vùng ảnh nằm trong bbox
        cropped_image = image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        # Chuyển mảng numpy thành mảng 2 chiều
        cropped_image_2d = np.squeeze(np.asarray(cropped_image))

        max_value = np.amax(cropped_image_2d)
        # Tính tỉ lệ chiều dài/ chiều rộng của biển số
        ratio = (bbox[2]-bbox[0])/(bbox[3]-bbox[1])
        if (ratio >= 2.2):
            type = 1
            cropped_image = cv2.resize(cropped_image, (256, 100))
        else:
            type = 2
            cropped_image = cv2.resize(cropped_image, (550, 100))
        
        read_fn = read(cropped_image, type)
        return type, read_fn, max_value, cropped_image

# Test xuất ra loại biển số (1 dòng/ 2 dòng)

In [10]:
# folder_path = "yellow_license_plate/" # đường dẫn đến folder chứa ảnh

# type = []
# for filename in os.listdir(folder_path):
#     if filename.endswith(".jpg") or filename.endswith(".png"): 
#         # kiểm tra xem file có phải là ảnh jpg hoặc png không
#         file_path = os.path.join(folder_path, filename)
#         img = cv2.imread(file_path) # đọc ảnh bằng OpenCV
#         # sử dụng ảnh tại đây
#         name, extension = os.path.splitext(filename) # Tách phần đuôi của tên tệp
#         print("Image: ", name)
#         pre_final.append((name, readPlate(img, model)))

# # Tạo DataFrame từ mảng
# df = pd.DataFrame(type, columns=['Image', 'Predicted'])

# # Xuất ra file CSV
# df.to_csv('predicted_output.csv', index=False)


# Test những biển số sai sót

In [17]:
img = cv2.imread("aaa/test9.jpg") # đọc ảnh bằng OpenCV

readPlate(img, model)

(2,
 '18A21443',
 255,
 array([[[  6,  23,  35],
         [  7,  24,  35],
         [  9,  24,  35],
         ...,
         [ 77,  84,  95],
         [ 78,  85,  95],
         [ 79,  86,  96]],
 
        [[  7,  24,  34],
         [ 10,  27,  35],
         [ 10,  27,  35],
         ...,
         [114, 115, 124],
         [130, 130, 135],
         [155, 155, 159]],
 
        [[ 10,  28,  36],
         [ 10,  27,  35],
         [ 11,  28,  36],
         ...,
         [107, 105, 112],
         [114, 113, 116],
         [121, 120, 123]],
 
        ...,
 
        [[ 45,  53,  55],
         [ 43,  50,  53],
         [ 44,  50,  53],
         ...,
         [ 38,  46,  61],
         [ 35,  43,  57],
         [ 32,  40,  54]],
 
        [[ 44,  54,  56],
         [ 40,  50,  52],
         [ 40,  49,  51],
         ...,
         [ 30,  39,  54],
         [ 32,  41,  56],
         [ 34,  43,  58]],
 
        [[ 50,  58,  61],
         [ 47,  55,  57],
         [ 47,  54,  57],
         ...,
     

# xuất ra file csv kết quả kiểm thử hình ảnh trong tập kiểm thử

In [25]:
folder_path = "./test_set/" # đường dẫn đến folder chứa ảnh

pre_final = []
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        # kiểm tra xem file có phải là ảnh jpg hoặc png không
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path) # đọc ảnh bằng OpenCV
        # sử dụng ảnh tại đây
        name, extension = os.path.splitext(filename) # Tách phần đuôi của tên tệp
        # print(readPlate(img, model))
        type, pre_fn, a, b = readPlate(img, model)
        pre_final.append((name, type, pre_fn))

# Tạo DataFrame từ mảng
df = pd.DataFrame(pre_final, columns=['Image', 'Type', 'Predicted'])

# Xuất ra file CSV
df.to_csv('test_set/predicted_testset_output.csv', index=False)


# đọc video kiểm thử 

In [18]:
# Tạo một đối tượng VideoCapture để đọc video
cap = cv2.VideoCapture('test (1).MOV')

# Lấy kích thước khung hình của video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Tạo đối tượng VideoWriter để ghi video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('processed_video.mp4', fourcc, 30, (width, height))
# Đọc từng khung hình của video, xử lý và ghi lại
result_s = ''
r = 0

with open("license_plates_history.txt", "w") as file:
    file.write('')

max_value_before = 0
id = 1
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Xử lý khung hình ở đây
    processed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(processed_frame)
    if len(results.pred[0]) >= 1:
        if (results.pred[0][0][0] <= 10 and r == 1):
            r = 0
        for i in range (len(results.pred[0])):
            accuracy = float(results.pred[0][i][4].item())
            
            if (accuracy >= 0.7 and (results.pred[0][i][2].item() - results.pred[0][i][0].item()) >= 100 and r == 0 and results.pred[0][i][2] < 650 and results.pred[0][0][0] >= 10):
                if (readPlate(processed_frame,model) != None):
                    k, result_s, max_value, cropped_image = readPlate(processed_frame,model)
                if (result_s != ""):
                    r = 1
                    cv2.imwrite(f"{id}.jpg", cropped_image)
                    with open('license_plates_history.txt', 'a') as file:
                        file.write('{} {}\n'.format(id, result_s))
                        id = id + 1
    else:
        r = 0

    # Vẽ chuỗi result_s lên khung hình results
    cv2.putText(results.render()[0], result_s, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
    if (r == 0):
        cv2.putText(results.render()[0], "Next license plate!", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
    # Ghi khung hình đã xử lý vào đối tượng VideoWriter
    out.write(cv2.cvtColor(results.render()[0], cv2.COLOR_RGB2BGR))

    # # Hiển thị khung hình đã xử lý
    cv2.imshow('Processed Frame', cv2.cvtColor(results.render()[0], cv2.COLOR_RGB2BGR))
    if cv2.waitKey(1) == ord('q'):
        break

# Giải phóng các tài nguyên và đóng các cửa sổ hiển thị
out.release()
cv2.destroyAllWindows()

In [34]:

# Đọc file CSV vào DataFrame
df = pd.read_csv('test_set/predicted_testset_output.csv')

# Sắp xếp DataFrame theo cột số
df_sorted = df.sort_values(by='Image', ascending=True)

# Lưu lại kết quả vào file CSV
df_sorted.to_csv('test_set/predicted_output.csv', index=False)

In [36]:
# Đọc hai file CSV và lưu chúng vào hai DataFrame
df1 = pd.read_csv('test_set/predicted_output.csv')
df2 = pd.read_csv('test_set/true_output.csv')

# Kết hợp hai DataFrame và lấy các cột cần thiết
merged_df = pd.merge(df1[['Image', 'Predicted']], df2[['Image', 'True']], on='Image')

merged_df['Diff'] = merged_df.apply(lambda row: 0 if row['Predicted'] == row['True'] else 1, axis=1)

# Thêm cột Predict dựa trên giá trị của cột thứ 2
merged_df['Predict'] = merged_df['Predicted'].apply(lambda x: 0 if x == "0" else 1)

# Thêm cột Miss để kiểm tra độ dài khác nhau giữa hai cột 'Predicted' và 'True'
merged_df['Miss'] = merged_df.apply(lambda row: 1 if (not pd.isnull(row['Predicted'])) and (not pd.isnull(row['True'])) and len(str(row['Predicted'])) - len(str(row['True'])) != 0 else 0, axis=1)

# Lưu các cột vào file CSV
merged_df.to_csv('test_set/Evaluate.csv', index=False)


In [37]:
# Đọc file CSV vào DataFrame
merged_df = pd.read_csv('test_set/Evaluate.csv')

zero_Predict_ratio = (merged_df['Predict'] == 0).sum() / len(merged_df) * 100
miss_Predict_ratio = (merged_df['Miss'] == 1).sum()/ len(merged_df) * 100
right_Predict_ratio = 100 - ((merged_df['Diff'] == 1).sum()) / len(merged_df) * 100
wrong_Predict = (merged_df['Diff'] == 1).sum()- (merged_df['Predict'] == 0 - (merged_df['Miss'] == 1).sum()).sum()
wrong_Predict_ratio = wrong_Predict / len(merged_df) * 100

print("Số ảnh kiểm thử: ", len(merged_df))
print("Tỉ lệ không thể nhận diện biển số: {:.2f}%".format(zero_Predict_ratio))
print("Tỉ lệ đọc sai biển số: {:.2f}%".format(wrong_Predict_ratio- miss_Predict_ratio))
print("Tỉ lệ đọc thiếu chữ số: {:.2f}%".format(miss_Predict_ratio))
print("Tỉ lệ biển số đọc đúng: {:.2f}%".format(right_Predict_ratio))

mask = (merged_df['Diff'] == 1)
df_filtered = merged_df.loc[mask]
df_filtered.head(50)

Số ảnh kiểm thử:  489
Tỉ lệ không thể nhận diện biển số: 0.00%
Tỉ lệ đọc sai biển số: 5.32%
Tỉ lệ đọc thiếu chữ số: 1.84%
Tỉ lệ biển số đọc đúng: 92.84%


,Image,Predicted,True,Diff,Predict,Miss
45,509,51A89711,51A89714,1,1,0
49,554,54Y210,54A72110,1,1,1
50,555,NaN,54A72110,1,1,0
57,647,51F24483,51F24403,1,1,0
67,782,30V39931,30V3993,1,1,1
68,783,30V39931,30V3993,1,1,1
82,1961,NaN,51A85325,1,1,0
134,3820,51B21666,51D21666,1,1,0
158,5347,51G25182,51G25181,1,1,0
198,6897,51Z210,51A72110,1,1,1
